In [1]:
import pandas as pd
import numpy as np
import wrangle as w

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 1000)

In [2]:
df = w.acquire_anonymized_curriculum_access_data()

In [3]:
df

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0
...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,Staff,2014-02-04,2014-02-04,2.0


In [4]:
# 1 - PHP
# 2 - Java
# 3 - Data Science
# 4 - Front End

In [5]:
df = w.clean_the_data(df)

## 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

**Findings:**
- There are 233 users that didn't access the curriculum during their cohort timeframe, but some of those don't have a cohort listed (78)
- I defined "hardly active" as those students who accessed less than or equal to 10% of the average usage (pages viewed)
- 54 users accessed less than or equal to 10% of the average number of pages viewed per user.
- All but one user was from the Java program, user_id 679 was data science(assumed quit the course)
- **Oberon** and **Sequoia** cohorts only had students access the curriculum up through the first month of the course. ***This might mean that these cohorts used a different curriculum platform or there is something wrong with the start and end dates in the db***
- user_ids 24, 41, 371, 741, 56, 57, 45, 36, 250, 65, 52, 33, 253, 278 only started accessing the curriculum when the cohort was almost finished 
- user_ids 879, 832,  388, 572, 772, 529, 543, 46, 13, 702, 806 accessed the curriculum on the first day (or first week) and then stopped. ***I assume these are students who dropped out.***



Below is the code that helped me discover these finidings.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        900223 non-null  datetime64[ns]
 1   time        900223 non-null  object        
 2   page        900222 non-null  object        
 3   user_id     900223 non-null  int64         
 4   cohort_id   847330 non-null  float64       
 5   ip          900223 non-null  object        
 6   name        847330 non-null  object        
 7   start_date  847330 non-null  datetime64[ns]
 8   end_date    847330 non-null  datetime64[ns]
 9   program_id  900223 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(1), object(5)
memory usage: 75.5+ MB


In [7]:
#only look at students
students = df[df['name'] != 'Staff']

In [8]:
#find those who accessed the curriculum during their class
active_student_access = students.query('start_date < date < end_date')

In [9]:
total_users = students.user_id.nunique()

In [10]:
active_users = active_student_access.user_id.nunique()

In [11]:
total_users - active_users
#interesting that there are 257 users that didn't access the curriculum during their cohort, 
#but some of those don't have a cohort listed

233

In [12]:
#those that don't have a cohort listed and therefore don't have a start and end date
df[df['cohort_id'].isnull()].user_id.nunique()

78

In [13]:
usage = active_student_access.user_id.value_counts().sort_values()

In [14]:
#I defined "hardly active" as those students who accessed less than or equal to 10% of the average usage (pages viewed)
usage.mean() * .10

89.00359116022099

In [15]:
#how many accessed less than 10% of the average -- 54
usage[usage <= 89]

879     1
679     3
832     3
278     4
956     5
812     7
388     8
572    12
889    16
968    16
487    17
963    17
961    18
772    20
49     25
64     25
976    25
24     26
41     28
972    31
973    32
967    33
529    35
959    38
971    41
964    42
371    42
981    42
960    45
741    47
978    49
965    51
56     52
962    52
543    53
46     54
57     62
969    62
858    63
45     65
36     65
975    66
250    67
65     73
52     75
977    76
974    76
13     79
33     79
955    80
702    85
806    88
954    88
253    89
Name: user_id, dtype: int64

In [16]:
#278 is weird because they only started accessing the curriculum when the cohort was almost over
#Those who started accessing the curriculum when the cohort was almost over
#Java students
#278, 812
active_student_access[active_student_access.user_id == 278]

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-09-27,2018-09-27,13:57:44,/,278,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:47:37,java-ii/arrays,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:58:48,java-ii/arrays,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:59:07,java-ii/collections,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java


In [17]:
#test oddities of those who didn't view many times during cohort
active_student_access[active_student_access.user_id == 24]

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-01-26,2018-01-26,11:24:17,appendix,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-26,2018-01-26,11:24:29,appendix/capstone-workbook,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,11:25:49,/,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,11:25:54,spring,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,11:25:59,spring/fundamentals/views,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,13:28:23,mkdocs/search_index.json,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,13:28:33,html-css,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,13:28:41,html-css/css-ii/bootstrap-introduction,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,13:29:50,/,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java


In [18]:
#24 seems to have completed the course but barely looked at the curriculum - Java

#858 started using a month plus in and then stopped after a week

#24, 41, 371, 741, 56, 57, 45, 36, 250, 65, 52, 33, 253, 278 only started accessing the curriculum when the cohort was almost finished - Java



In [19]:
#The below users accessed the curriculum on the first day (or first week) and then stopped. I assume these are students who dropped out.
#Java
#879, 832,  388, 572, 772, 529, 543, 46, 13, 702, 806,
#981 - listed as Neptune, but seems like it could have been Oberon

#Oberon Cohort(968, 963, 956, 961, 972, 973, 967, 959, 971, 964, 960, 978, 965, 962, 969, 975)
#No usage past 2021-04-21, but cohort end_date == 2021-10-01

#Data Science
#679, 

#487 only accessed the appendix pages one day during the first month of classes and that's it. Also, data science

In [20]:
active_student_access[active_student_access.name == 'Oberon'].user_id.value_counts()

966    159
970    113
957    112
979    104
958     95
954     88
955     80
977     76
974     76
975     66
969     62
962     52
965     51
978     49
960     45
964     42
971     41
959     38
967     33
973     32
972     31
976     25
961     18
963     17
968     16
956      5
Name: user_id, dtype: int64

In [21]:
active_student_access[active_student_access.name == 'Oberon'].end_date.max()

Timestamp('2021-10-01 00:00:00')

In [22]:
active_student_access[active_student_access.name == 'Oberon'].date.max()

Timestamp('2021-04-21 00:00:00')

In [23]:
active_student_access[active_student_access.name == 'Oberon'].start_date.min()

Timestamp('2021-04-12 00:00:00')

In [24]:
active_student_access[active_student_access.name == 'Oberon'].date.min()

Timestamp('2021-04-13 00:00:00')

In [25]:
active_student_access[active_student_access.name == 'Sequoia'].user_id.value_counts()

10    205
15    179
42    175
28    158
27    154
38    148
16    141
17    134
47    125
22    114
55     95
33     79
52     75
65     73
36     65
45     65
57     62
56     52
41     28
24     26
49     25
Name: user_id, dtype: int64

In [26]:
active_student_access[active_student_access.name == 'Sequoia'].end_date.max()

Timestamp('2018-02-15 00:00:00')

In [27]:
active_student_access[active_student_access.name == 'Sequoia'].date.max()

Timestamp('2018-02-14 00:00:00')

In [28]:
active_student_access[active_student_access.name == 'Sequoia'].start_date.min()

Timestamp('2017-09-27 00:00:00')

In [29]:
active_student_access[active_student_access.name == 'Sequoia'].date.min()

Timestamp('2018-01-26 00:00:00')

## 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [30]:
def one_user_df_prep(df, user):
    '''
    This function returns a dataframe consisting of data for only a single defined user
    '''
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages_one_user = df['page'].resample('d').count()
    return pages_one_user

In [31]:
def compute_pct_b(pages_one_user, span, weight, user):
    '''
    This function adds the %b of a bollinger band range for the page views of a single user's log activity
    '''
    # Calculate upper and lower bollinger band
    midband = pages_one_user.ewm(span=span).mean()
    stdev = pages_one_user.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    
    # Add upper and lower band values to dataframe
    bb = pd.concat([ub, lb], axis=1)
    
    # Combine all data into a single dataframe
    my_df = pd.concat([pages_one_user, midband, bb], axis=1)
    my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
    
    # Calculate percent b and relevant user id to dataframe
    my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

In [32]:
def find_anomalies(df, user, span, weight, plot=False):
    '''
    This function returns the records where a user's daily activity exceeded the upper limit of a bollinger band range
    '''
    
    # Reduce dataframe to represent a single user
    pages_one_user = one_user_df_prep(df, user)
    
    # Add bollinger band data to dataframe
    my_df = compute_pct_b(pages_one_user, span, weight, user)
    
    # Plot data if requested (plot=True)
    if plot:
        plot_bands(my_df, user)
    
    # Return only records that sit outside of bollinger band upper limit
    return my_df[my_df.pct_b>1]

In [33]:
span = 30
weight = 2

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [34]:
anomalies.sort_values(by='pages_one_user', ascending = False)

,pages_one_user,midband,ub,lb,pct_b,user_id
date,,,,,,
2020-07-22,285,44.915959,192.499064,-102.667146,1.313386,11
2019-03-03,272,24.721632,163.040772,-113.597508,1.393869,341
2020-04-16,271,44.194413,193.288101,-104.899276,1.260614,570
2019-07-25,253,34.698961,160.039466,-90.641543,1.370832,422
2020-04-26,234,54.999406,211.467475,-101.468663,1.072004,570
...,...,...,...,...,...,...
2020-09-25,1,0.064516,0.564257,-0.435225,1.435969,406
2018-05-04,1,0.083575,0.654077,-0.486926,1.303175,84
2020-07-09,1,0.066557,0.670070,-0.536955,1.273342,358


- user 11 was a student and then on staff

In [35]:
df[df.name == 'Hampton']

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-01-26,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,10:40:15,javascript-i/functions,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
...,...,...,...,...,...,...,...,...,...,...
2020-04-23,2020-04-23,10:18:07,content/javascript/arrays/gitbook/images/favic...,646,8.0,173.174.197.146,Hampton,2015-09-22,2016-02-06,PHP
2020-04-23,2020-04-23,10:18:47,content/javascript/arrays/iterating.html,646,8.0,173.174.197.146,Hampton,2015-09-22,2016-02-06,PHP
2020-04-23,2020-04-23,10:19:24,content/javascript/functions.html,646,8.0,173.174.197.146,Hampton,2015-09-22,2016-02-06,PHP


In [36]:
df.groupby(['program_id'])['page'].nunique()

program_id
Data Science     682
Front End          4
Java            1913
PHP              710
nan             1112
Name: page, dtype: int64

In [37]:
# let's observe unique hits based on cohort
cohort_counts = df.groupby(['name','program_id'])['user_id', 'date','page'].nunique()

In [38]:
cohort_counts

,,user_id,date,page
name,program_id,,,
Andromeda,Java,28,447,366
Apex,Java,24,386,273
Apollo,Front End,1,1,4
Arches,PHP,18,278,342
Badlands,PHP,3,12,44
Bash,Java,23,252,275
Bayes,Data Science,23,512,533
Betelgeuse,Java,22,430,334
Ceres,Java,29,466,301


In [39]:
df[df.user_id == 341].ip.value_counts()

97.105.19.58       677
172.124.70.146     462
173.174.243.231    233
204.44.112.76      180
70.123.203.131      25
192.171.117.210      4
Name: ip, dtype: int64

In [40]:
df.ip.value_counts()

97.105.19.58       284579
97.105.19.61        61662
192.171.117.210      9515
71.150.217.33        6791
76.185.145.231       4754
                    ...  
172.58.109.140          1
99.203.213.59           1
99.203.212.173          1
173.239.198.246         1
72.181.124.244          1
Name: ip, Length: 5531, dtype: int64

In [41]:
ip_df = pd.DataFrame(df.ip.value_counts(dropna=False)).reset_index().\
                rename(columns={'index': 'ip', 'ip': 'count'})
ip_df.head()

,ip,count
0,97.105.19.58,284579
1,97.105.19.61,61662
2,192.171.117.210,9515
3,71.150.217.33,6791
4,76.185.145.231,4754


In [42]:
# calculate probabity for each ip 

# ip_prob = count for each ip / total count in the dataframe

ip_df2 = pd.DataFrame((df.ip.value_counts(dropna=False))/df.ip.count()).reset_index().\
                rename(columns={'index': 'ip', 'ip': 'proba'})
ip_df2.head()

,ip,proba
0,97.105.19.58,0.316121
1,97.105.19.61,0.068496
2,192.171.117.210,0.010570
3,71.150.217.33,0.007544
4,76.185.145.231,0.005281


In [43]:
# merge the two data frames create above into a single one:
ip_df = ip_df.merge(ip_df2, on='ip')

In [44]:
ip_df.tail()

,ip,count,proba
5526,172.58.109.140,1,0.000001
5527,99.203.213.59,1,0.000001
5528,99.203.212.173,1,0.000001
5529,173.239.198.246,1,0.000001
5530,72.181.124.244,1,0.000001


## 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

**Findings:**
- The first DS cohort listed (Bayes) graduated in 



Below is the code that helped me discover these finidings.

In [45]:
df[df.program_id == 'Data Science'].name.value_counts()

Darden      32015
Bayes       26538
Curie       21582
Easley      14715
Florence     8562
Name: name, dtype: int64

In [46]:
ds_accessed_pages = list(students[students['program_id'] == 'Data Science'].page.unique())

In [47]:
webdev_accessed_pages = list(students[(students['program_id'] == 'Java') | (students['program_id'] == 'PHP')].page.unique())

In [48]:
both_access = [page for page in ds_accessed_pages if page in webdev_accessed_pages]

In [51]:
both_df = students[students['page'].isin(both_access)]

In [52]:
both_df['2019':].program_id

date
2019-01-01     PHP
2019-01-01    Java
2019-01-01    Java
2019-01-01    Java
2019-01-01    Java
              ... 
2021-04-21    Java
2021-04-21    Java
2021-04-21    Java
2021-04-21    Java
2021-04-21    Java
Name: program_id, Length: 216445, dtype: object

In [50]:
ds_accessed_pages

['/',
 '3-sql/1-mysql-overview',
 '2-storytelling/bad-charts',
 '2-storytelling/misleading1_baseball.jpg',
 '2-storytelling/misleading1_fox.jpg',
 '2-storytelling/misleading3_deaths.jpg',
 'appendix/cli-git-overview',
 '1-fundamentals/1.1-intro-to-data-science',
 '1-fundamentals/modern-data-scientist.jpg',
 '1-fundamentals/AI-ML-DL-timeline.jpg',
 '1-fundamentals/1.2-data-science-pipeline',
 '1-fundamentals/DataToAction_v2.jpg',
 'search/search_index.json',
 '13-advanced-topics/3.7-styling-webpages',
 '1-fundamentals/1.3-pipeline-demo',
 '1-fundamentals/2.1-excel-overview',
 '3-vocabulary.md',
 '6-regression/1-overview',
 '10-anomaly-detection/1-overview',
 '10-anomaly-detection/AnomalyDetectionCartoon.jpeg',
 '11-nlp/1-overview',
 '6-regression/2-regression-excel',
 '6-regression/3.1-acquire-and-prep',
 '6-regression/3.2-split-and-scale',
 '6-regression/3.3-explore',
 '6-regression/3.4-feature-engineering',
 '6-regression/3.5-modeling',
 '6-regression/3.6-evaluation',
 '6-regression/e